# BC2- Predicting Cancellations 

In [1]:
import flask
import pickle

In [2]:
import os
from pathlib import Path
from math import ceil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pandas_profiling import ProfileReport




from joblib import dump

from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report, confusion_matrix, precision_score
from sklearn.neighbors import LocalOutlierFactor
from sklearn.neural_network import MLPClassifier




import graphviz
import imblearn

import warnings
warnings.filterwarnings('ignore')

# Defining PROJECT_ROOT
PROJECT_ROOT = Path(os.path.abspath('')).resolve().parents[0]

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
from pandas import set_option
from pandas.plotting import scatter_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm




In [ ]:
import matplotlib.pyplot as plt 
from mlxtend.plotting import plot_confusion_matrix 
from mlxtend.classifier import StackingClassifier 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB  
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('H2.csv')

In [ ]:
df.shape

### Cleaning

In [ ]:
df.drop(['Country'], axis=1,inplace = True)
# Since you dont have this information prior to the check-in

df.drop(['IsCanceled'], axis=1,inplace = True)
#Since we will try to predict also the not show clients

df.drop(['DepositType'], axis=1, inplace = True)

df.drop(columns = ['ArrivalDateYear','ArrivalDateMonth','ArrivalDateDayOfMonth','ArrivalDateWeekNumber'], axis=1, inplace = True)


In [ ]:
#Cleaning the target variable
df['ReservationStatus'] =df['ReservationStatus'].astype('category').cat.codes

df['ReservationStatus'].value_counts()

### Check continuos correlations 

In [ ]:

cor = df[['ReservationStatus','LeadTime','StaysInWeekendNights','StaysInWeekNights',
          'Adults','Children','Babies','PreviousCancellations',
          'PreviousBookingsNotCanceled','BookingChanges','DaysInWaitingList','ADR',
         'RequiredCarParkingSpaces','TotalOfSpecialRequests']].corr()


In [ ]:
def cor_heatmap(cor):
    plt.figure(figsize=(13,7))
    sns.heatmap(data = cor, annot = True, cmap = 'YlGnBu', fmt = '.1')
    plt.show
    

cor_heatmap(cor)

### Check categorical correlation 

In [ ]:
#ax = sns.countplot(x="Agent", hue="ReservationStatus", data=df[df['Agent']=='          9'])


In [ ]:
#ax = sns.countplot(x="ArrivalDateWeekNumber", hue="ReservationStatus", data=df)



In [ ]:
df.describe().T

In [ ]:
df.describe(include='all').transpose()

# Outlier removal

In [ ]:
#fill missing data in Children
df['Children'].fillna(df['Children'].median(), inplace=True)
df['Children'] = df['Children'].astype(int)

In [ ]:
non_metric_features= ['AssignedRoomType','CustomerType',
                      'DistributionChannel','IsRepeatedGuest','MarketSegment','Meal',
                      'ReservationStatusDate','ReservedRoomType','Agent','Company']
metric_features = df.columns.drop(non_metric_features)

### Spliting the data

In [ ]:
X = df.drop(columns = ['ReservationStatus'])
y = df[['ReservationStatus']]



In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=15
                                                    , stratify=y)

combine = [X_train,X_test]

### Numerical
- Lead time- nao apagar
- StaysInWeekendNights - apagar o 14
- StaysInWeekNights - apagar o 24
- Adults - nao apagar
- Children - nao apagar
- Babies - apagar o 10
- PreviousCancellations - apagar no 10
- PreviousBookingsNotCanceled - apagar no 34
- BookingChanges - apagar no 12
- DaysInWaitingList - apagar no 260
- ADR - apagar no 400 
- RequiredCarParkingSpaces - Nao apagar
- TotalOfSpecialRequests - Nao apagar

**13 numerical variables**

In [ ]:
# hist = px.histogram(X_train, x=X_train['LeadTime'], color_discrete_sequence=['darkseagreen'], template='plotly_white')
# hist.show()

In [ ]:
Xy = pd.concat([X_train, y_train], axis=1)

In [ ]:
lof = LocalOutlierFactor(n_neighbors=20, contamination = 0.03)
result=lof.fit_predict(Xy[metric_features])

In [ ]:
outliers_index= np.where(result==-1)
outliers_index=outliers_index[0]


In [ ]:
metric_features_1 = Xy[metric_features].drop(Xy[metric_features].index[outliers_index],axis=0)

df_2 = Xy.drop(Xy.index[outliers_index],axis=0)



In [ ]:
print('Percentage of data kept after removing outliers:'
      , np.round(metric_features_1.shape[0] /Xy[metric_features].shape[0], 4))

In [ ]:
Xy = df_2

In [ ]:
X_train = Xy.iloc[:,:-1]
y_train = Xy.iloc[:,-1]

### Categorical

Eu acho que nestas temos de ver é como é que reduzimos o numero de categorias

- ArrivalDateYear - Nao apagar
- ArrivalDateMonth - Nao apagar
- ArrivalDateWeekNumber - Nao apagar
- ArrivalDateDayOfMonth - Nao apagar
- Meal - Nao apagar / FB
- MarketSegment - Undefined
- DistributionChannel - Undefined and GDS
- IsRepeatedGuest - variavel binária
- ReservedRoomType - Juntar as categorias
- AssignedRoomType - Igual ao de cima
- Agent - Nao apagar
- Company - Nao apagar
- CustomerType - Nao apagar

**13 categorical**

In [ ]:
# hist = px.histogram(X_train, x=X_train['Agent'], color_discrete_sequence=['darkseagreen'], template='plotly_white')
# hist.show()

In [ ]:
#ax = sns.countplot(x="Meal", hue="ReservationStatus", data=X_train)


0 - cancelar
1 - check out
2 - no show

# Feature Engineering

In [ ]:
combine = [X_train,X_test]

In [ ]:
 months = {
            'January' : 1,
            'February' : 2,
            'March' : 3,
            'April' : 4,
            'May' : 5,
            'June' : 6,
            'July' : 7,
            'August' : 8,
            'September' : 9, 
            'October' : 10,
            'November' : 11,
            'December' : 12
    }

In [ ]:
# for dataset in combine:
#     dataset['ArrivalDateMonth'] = dataset['ArrivalDateMonth'].map(months)

In [ ]:
for dataset in combine:
    dataset["Meal"] = dataset["Meal"].map({"BB       ":'BB',"SC       ":'SC','FB       ':'HB/FB', 'HB       ':'HB/FB'})




In [ ]:
for dataset in combine:
    dataset['MarketSegment'] = np.where(dataset['MarketSegment'] == 'Undefined','Other', dataset['MarketSegment'])
    dataset['MarketSegment'] = np.where(dataset['MarketSegment'] == 'Aviation','Other', dataset['MarketSegment']) 
    dataset['MarketSegment'] = np.where(dataset['MarketSegment'] == 'Complementary','Other', dataset['MarketSegment']) 



    

In [ ]:
for dataset in combine:
    dataset['DistributionChannel'] = np.where(dataset['DistributionChannel'] == 'Undefined','Other', dataset['DistributionChannel'])
    dataset['DistributionChannel'] = np.where(dataset['DistributionChannel'] == 'GDS','Other', dataset['DistributionChannel']) 



In [ ]:
for dataset in combine:
    dataset['Agent'] = np.where(dataset['Agent'] != '       NULL','1', dataset['Agent'])
    dataset['Agent'] = np.where(dataset['Agent'] == '       NULL','0', dataset['Agent'])





In [ ]:
for dataset in combine:
    dataset['Company'] = np.where(dataset['Company'] != '       NULL','1', dataset['Company'])
    dataset['Company'] = np.where(dataset['Company'] == '       NULL','0', dataset['Company'])



In [ ]:
for dataset in combine:
    dataset['Nr People'] = dataset['Adults'] + dataset['Children'] + dataset['Babies']



In [ ]:
for dataset in combine:
    X_train['Totalnights']= X_train.StaysInWeekNights + X_train.StaysInWeekendNights
    X_test['Totalnights']= X_test.StaysInWeekNights + X_test.StaysInWeekendNights

# Normalization

In [ ]:
metric_features = ['LeadTime', 'StaysInWeekendNights', 'StaysInWeekNights', 'Adults',
       'Children', 'Babies', 'PreviousCancellations',
       'PreviousBookingsNotCanceled', 'BookingChanges', 'DaysInWaitingList',
       'ADR', 'RequiredCarParkingSpaces', 'TotalOfSpecialRequests']

In [ ]:
# scaler1 = StandardScaler()
# scaled_feat_X_train = scaler1.fit_transform(X_train[metric_features])
# X_train_scaled = pd.DataFrame(scaled_feat_X_train, columns =X_train[metric_features].columns )



In [ ]:
# scaler2 = StandardScaler()
# scaled_feat_X_test = scaler2.fit_transform(X_test[metric_features])
# X_test_scaled = pd.DataFrame(scaled_feat_X_test, columns =X_test[metric_features].columns )



# Feature Enconding

In [ ]:
# One hot encode categorical features
ohc_features = [#'ArrivalDateYear', 'ArrivalDateMonth', 'ArrivalDateWeekNumber', 'ArrivalDateDayOfMonth'
                'Meal', 
                'MarketSegment', 'DistributionChannel','IsRepeatedGuest','ReservedRoomType','AssignedRoomType',
               'Agent','Company','CustomerType']

ohc = OneHotEncoder(sparse=False, drop= 'first')
df_ohc_train = pd.DataFrame(ohc.fit_transform(X_train[ohc_features]),
index=X_train.index,
columns=ohc.get_feature_names(ohc_features))
#X_train  = pd.concat([X_train,df_ohc_train],axis = 1)

In [ ]:
ohc = OneHotEncoder(sparse=False,drop= 'first' )
df_ohc_test = pd.DataFrame(ohc.fit_transform(X_test[ohc_features]),
index=X_test.index,
columns=ohc.get_feature_names(ohc_features))
#X_test  = pd.concat([X_test,df_ohc_test],axis = 1)

In [ ]:
X_train.reset_index(inplace = True, drop = True)
X_test.reset_index(inplace = True, drop = True)
df_ohc_test.reset_index(inplace = True, drop = True)
df_ohc_train.reset_index(inplace = True, drop = True)



In [ ]:
# X_train[metric_features] = X_train_scaled
# X_test[metric_features] = X_test_scaled

In [ ]:
X_train  = pd.concat([X_train,df_ohc_train],axis = 1)
X_test  = pd.concat([X_test,df_ohc_test],axis = 1)

In [ ]:
X_train.drop(columns = ohc_features,inplace = True)
X_test.drop(columns = ohc_features,inplace = True)

X_test.drop('ReservationStatusDate',axis = 1, inplace = True)
X_train.drop('ReservationStatusDate',axis = 1, inplace = True)





# Model Selection

In [ ]:
# # Spot-Check Algorithms
# def GetBasedModel():
#     basedModels = []
#     basedModels.append(('LR'   , LogisticRegression())) 
#     basedModels.append(('LDA'  , LinearDiscriminantAnalysis()))
#     basedModels.append(('KNN'  , KNeighborsClassifier()))
#     basedModels.append(('CART' , DecisionTreeClassifier()))
#     basedModels.append(('NB'   , GaussianNB()))
#     basedModels.append(('AB'   , AdaBoostClassifier()))
#     basedModels.append(('GBM'  , GradientBoostingClassifier()))
#     basedModels.append(('RF'   , RandomForestClassifier()))
#     basedModels.append(('ET'   , ExtraTreesClassifier()))

    
#     return basedModels

In [ ]:
# def BasedLine2(X_train, y_train,models):
#     # Test options and evaluation metric
#     num_folds = 10
#     scoring = 'accuracy'

#     results = []
#     names = []
#     for name, model in models:
#         kfold = StratifiedKFold(n_splits=num_folds, random_state=SEED)
#         cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
#         results.append(cv_results)
#         names.append(name)
#         msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
#         print(msg)
        
#     return names, results

In [ ]:
# class PlotBoxR(object):
    
    
#     def __Trace(self,nameOfFeature,value): 
    
#         trace = go.Box(
#             y=value,
#             name = nameOfFeature,
#             marker = dict(
#                 color = 'rgb(0, 128, 128)',
#             )
#         )
#         return trace

#     def PlotResult(self,names,results):
        
#         data = []

#         for i in range(len(names)):
#             data.append(self.__Trace(names[i],results[i]))


#         py.iplot(data)

In [ ]:
# SEED = 7
# np.random.seed(SEED)

In [ ]:
# models = GetBasedModel()
# names,results = BasedLine2(X_train, y_train,models)
# #PlotBoxR().PlotResult(names,results)

### Model Finetuning

In [ ]:
# # Define the Random Forest model
# rf = RandomForestClassifier(random_state=0,class_weight='auto')

# # Define the grid to explore
# grid = {
#     "criterion": ["entropy", "gini"],
#     "max_depth": [3, 6, 9],
#     "min_samples_split": [0.005, 0.01, 0.05],
#     "max_features": [None, "sqrt"],
#     "class_weight": [None, 'balanced', {0: 1, 1: 2}]
#     }

In [ ]:
# # Instatiating GridSearch
# splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
# gscv = GridSearchCV(rf, grid, cv=splitter, scoring='f1_weighted', refit=True, verbose=-1, n_jobs=-1)

# # Get ID of grid search
# # id_num = input("Insert GridSearch ID number: ")
# id_num = 1

In [ ]:
# # Grid Search and model training
# #gscv.fit(X_train, y_train)

# # Saving cv_results for specific Grid Search run
# #out_path = os.path.join(PROJECT_ROOT, 'analysis')
# #score_summary = pd.DataFrame(gscv.cv_results_).sort_values(by="mean_test_score", ascending=False)
# #score_summary.to_csv(os.path.join(out_path, 'grid_search_results{}.csv'.format(id_num)))

# # Saving best model
# best_rf = gscv.best_estimator_
# dump(best_rf, os.path.join(out_path, 'best_random_forest{}.joblib'.format(id_num)))

# # Get test set y_pred and evaluate on precision
# y_test_pred = best_rf.predict(X_test)

# print("The best model has a mean cross-validated precision of {0:.3f} and a test set precision of {1:.3f}".\
#       format(gscv.best_score_, precision_score(y_test, y_test_pred,average = 'micro')))
# print("\nThe hyper-parameters selected are:\n", gscv.best_params_)

In [ ]:
# print("The best model has a mean cross-validated precision of {0:.3f} and a test set precision of {1:.3f}".\
#       format(gscv.best_score_, precision_score(y_test, y_test_pred,average = 'micro')))
# print("\nThe hyper-parameters selected are:\n", gscv.best_params_)

# Feature Selection

In [ ]:
X_train= X_train[['LeadTime', 'StaysInWeekendNights', 'StaysInWeekNights', 'Adults',
       'Children', 'Babies', 'PreviousCancellations',
       'PreviousBookingsNotCanceled', 'BookingChanges', 'DaysInWaitingList',
       'ADR', 'RequiredCarParkingSpaces', 'TotalOfSpecialRequests',
       'Nr People', 'Totalnights', 'Meal_HB/FB', 'Meal_SC',
       'MarketSegment_Direct', 'MarketSegment_Groups',
       'MarketSegment_Offline TA/TO', 'MarketSegment_Online TA',
       'MarketSegment_Other', 'DistributionChannel_Direct',
       'DistributionChannel_Other', 'DistributionChannel_TA/TO',
       'IsRepeatedGuest_1','Agent_1', 'Company_1',
       'CustomerType_Group', 'CustomerType_Transient',
       'CustomerType_Transient-Party']]

In [ ]:
X_test= X_test[['LeadTime', 'StaysInWeekendNights', 'StaysInWeekNights', 'Adults',
       'Children', 'Babies', 'PreviousCancellations',
       'PreviousBookingsNotCanceled', 'BookingChanges', 'DaysInWaitingList',
       'ADR', 'RequiredCarParkingSpaces', 'TotalOfSpecialRequests',
       'Nr People', 'Totalnights', 'Meal_HB/FB', 'Meal_SC',
       'MarketSegment_Direct', 'MarketSegment_Groups',
       'MarketSegment_Offline TA/TO', 'MarketSegment_Online TA',
       'MarketSegment_Other', 'DistributionChannel_Direct',
       'DistributionChannel_Other', 'DistributionChannel_TA/TO',
       'IsRepeatedGuest_1','Agent_1', 'Company_1',
       'CustomerType_Group', 'CustomerType_Transient',
       'CustomerType_Transient-Party']]

### Oversampling the train dataset

In [ ]:
y_train.value_counts()

In [ ]:
# sampling = {0:36124,
#             1:36124,
#             2:9000
# }

In [ ]:
# from imblearn.over_sampling import SMOTE

# smote = SMOTE(sampling_strategy=sampling, k_neighbors = 5)
# X_sm, y_sm = smote.fit_sample(X_train, y_train)

# X_train = X_sm
# y_train = y_sm

In [ ]:
# y_train.value_counts()

In [ ]:
# y_test['ReservationStatus'].value_counts()

### Build and assess model

In [ ]:
rf = RandomForestClassifier(criterion='entropy', max_depth=200, max_features=None,
                       min_samples_split=0.005, random_state=0)
rf.fit(X_train, y_train)
RF_score = rf.score(X_test,y_test)
RF_score

In [ ]:
RF_score = rf.score(X_train,y_train)
RF_score


In [ ]:
# Get test set y_pred and evaluate on precision
y_test_pred = rf.predict(X_test)

### Model evaluation

In [ ]:
# Test set classification report
print(classification_report(y_test, y_test_pred))

In [ ]:
# Test set confusion matrix
pd.DataFrame(data=confusion_matrix(y_test, y_test_pred), 
             index=pd.Index(y_train.unique(), name="True"), 
             columns=pd.Index(y_train.unique(), name="Pred"))

In [ ]:
# plot confusion matrix
import itertools
 
def plot_confusion_matrix(cm, classes, normalize = False,
                          title='Confusion matrix',
                          cmap=plt.cm.Greens): # can change color 
    plt.figure(figsize = (10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, size = 24)
    plt.colorbar(aspect=4)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, size = 14)
    plt.yticks(tick_marks, classes, size = 14)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    # Label the plot
    for i, j in itertools.product(range(cm.shape[0]),   range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), 
             fontsize = 20,
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")
    plt.grid(None)
    plt.tight_layout()
    plt.ylabel('True label', size = 18)
    plt.xlabel('Predicted label', size = 18)
    
    

cm = confusion_matrix(y_test, y_test_pred)
plot_confusion_matrix(cm, classes = ['0 - Cancelled', '1 - Check-out', '2 - No show '],
                      title = 'Prediction Confusion Matrix')

In [ ]:
# Decision Tree Feature Importance plot
sns.set()

# data
feature_names = X_train.columns
feature_importances = pd.Series(rf.feature_importances_, index=feature_names).\
    sort_values(ascending=False)

# figure
fig = plt.figure(figsize=(19,5))

# axis
plt.bar(x=feature_importances.index, height=feature_importances.values)

# properties
plt.ylabel("Gini importance")  # The importance of a feature is computed as the (normalized) total reduction of the 
                               # criterion brought by that feature. It is also known as the Gini importance.
plt.xticks(rotation=90)
plt.title("Decision Tree Feature Importance", fontsize=18)

plt.show()

In [ ]:
import pickle
pickl = {'model': rf}
pickle.dump( pickl, open( 'model_file' + ".p", "wb" ) )